In [1]:
%gui qt
import time, sys

import numpy as np
np.set_printoptions(linewidth=130)

%matplotlib notebook
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [8, 6]

import ipywidgets as widgets
from IPython.display import display

from jupyterplot import ProgressPlot # see https://github.com/lvwerra/jupyterplot

import pyqtgraph as pg
from pyqtgraph.Qt import QtCore, QtGui

from sympy import Matrix
from scipy.spatial.transform import Rotation
import scipy

# imports from Python files (for brevity)
from teensy_imu import IMU
from ekf import EKF

r2d = 180 / np.pi
d2r = np.pi / 180

In [4]:
class UKF():
    def __init__(self, imu, dt, Qc, sa2, sm2, g, r):
        self.g = g
        self.r_m = r
        self.imu = imu
        self.dt = dt
        self.Qc = Qc
        
        # Initializes P, Q, and R matrices
        self.P = np.identity(7)
        self.Q = np.identity(7)*Qc
        R = np.hstack((np.identity(3)*sa2, np.zeros((3,3))))
        self.R = np.vstack((R, np.hstack((np.zeros((3,3)),(np.identity(3)*sm2)))))
        
        self.W = np.identity(7)*0.1
        
        # Initial state guess
        self.x = np.array([[1], [0], [0], [0], [0], [0], [0]])
        self.x_hat = np.array([[1], [0], [0], [0], [0], [0], [0]])
        
        self.n = self.x_hat.shape[0]
    
    def calculate_sigma_points(self):
        n = self.n
        self.sp = np.zeros((n, 2*n+1))
        self.sp[:,0] = self.x_hat[:,0]

        for i in range(1, n+1):
            self.sp[:,i] = self.x_hat[:,0] + scipy.linalg.sqrtm(n*self.Q)[:,i-1]
            self.sp[:,n+1] = self.x_hat[:,0] - scipy.linalg.sqrtm(n*self.Q)[:,i-1]
    
    def omega(self, q):
        """
        Rotation by an angular velocity
        """
        wx, wy, wz = q[4:]
        
        theta = np.sqrt(wx**2 + wy**2 + wz**2) + 1e-12 # small disturbance to avoid 0 values
        
        return np.array([
            [np.cos(theta/2)],
            [np.sin(theta/2)*(wx/theta)],
            [np.sin(theta/2)*(wy/theta)],
            [np.sin(theta/2)*(wz/theta)],
            
        ])
    
    def vector_rotation_by_quaternion(self, v):
        
    
    def f(self):
        """
        Passes the sigma points through the function of equations 3-17 and 3-26
        Write better comments if this shit actually works
        """
        self.xk = np.zeros(np.shape(self.sp))
        for i in range(1, np.shape(self.xk)[1]):
            om = self.omega(self.sp[:,i])
            self.xk[:4,i] = self.sp[:4,i] @ om
            
    def h_quaternion(self):
        """
        Measurement Model
        Takes in sigma points, generates the sigma point after being propagted through the
        nonlinear measurement model
        """
        y_sp = np.zeros(np.shape(self.sp))
        display(Matrix(self.sp))
        
        for i in range(np.shape(self.sp)[1]):
            attitude = self.sp[:,i]
            invq = np.array([[attitude[0], -attitude[1], -attitude[2], -attitude[3]]]).T
            display(Matrix(invq))
            
            # Accelerometer
            
            
            
    
    def predict_with_quaternion_ang_vec_model(self):
        """
        NOTE: NO LONGER JUST PREDICTION STEP
        Prediction step of UKF with quaternion + angular velocity model, state space is
        x = [q0 q1 q2 q3 omega_x omega_y omeg_z].T

        Inputs:
        u_t: commanded input to prediction step
        dt: delta time between updates
        xhat: mean
        P: posteriori covariance
        """

        # Compute sigma points for given mean ond posteriori covariance
        self.calculate_sigma_points()
        
        # Propagate sigma points
        uq = self.f()

        # Using propagated sigma points to approximate new xkm and Qkm
        w = 1/(2*self.n+1)
        xkm = np.sum(w*self.xk,axis=1)
        Qkm = np.zeros((self.n, self.n)) # estimated from sigma points
        
        for i in range(2*self.n+1):
            Qkm += w*(self.xk[:,i]-xkm).reshape(self.n,1)@(self.xk[:,i]-xkm).reshape(1,self.n)
        Qkm += self.W
        self.Qkm = Qkm
        
    def update_with_quaternion_ang_vec_model(self, z):
        self.h_quaternion()

In [5]:
g = np.array([[0, 0, 1]]).T # north east up
theta = 66*r2d # approximate magnetic inclination
r_m = np.array([[0, np.cos(theta), -np.sin(theta)]]).T

# sampling time
Fs = 20
dt = 1./200

# quaternion [q0 q1 q2 q3 omega_x omega_y omega_z].T
# omegas are angular velocity
x = np.array([[1], [0], [0], [0], [0], [0], [0]])

Qc = 0.01
sa2 = 0.5**2 #sigma_a^2
sm2 = 0.8**2 #sigma_m^2

# set up the live plotting
# plot = ProgressPlot(line_names=['roll', 'pitch', 'yaw'], y_lim=[-90, 90])

# create an IMU object to access data in real-time
imu = IMU(Fs)
time.sleep(0.1)

# create an EKF
ukf = UKF(imu, dt, Qc, sa2, sm2, g, r_m)

for i in range(2):
    print("STARTING")
    # Get normalized measurements
    Ax, Ay, Az = imu.get_acc()
    p, q, r = imu.get_gyr()
    mx, my, mz = imu.get_mag()

    # Calculate roll, pitch, and yaw
    # Predict step with UKF's quaternion angle vector model
    ukf.predict_with_quaternion_ang_vec_model()
    
    
    # Concatenate gyroscope, accelerometer, and magnetometer for measurement
    z = np.array([[Ax, Ay, Az], [p, q, r], [mx, my, mz]])
    
    # Update the step UKF with measurements
    ukf.update_with_quaternion_ang_vec_model(z)
    

STARTING


Matrix([
[1.0, 1.26457513110646,               1.0,               1.0,               1.0,               1.0,               1.0,               1.0,                1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
[0.0,              0.0, 0.264575131106459,               0.0,               0.0,               0.0,               0.0,               0.0,                0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
[0.0,              0.0,               0.0, 0.264575131106459,               0.0,               0.0,               0.0,               0.0,                0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
[0.0,              0.0,               0.0,               0.0, 0.264575131106459,               0.0,               0.0,               0.0,                0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
[0.0,              0.0,               0.0,               0.0,               0.0, 0.264575131106459,               0.0,               0.0,                0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
[0.0,              0.0,               0.0,    

Matrix([
[1.0],
[0.0],
[0.0],
[0.0]])

Matrix([
[1.26457513110646],
[             0.0],
[             0.0],
[             0.0]])

Matrix([
[               1.0],
[-0.264575131106459],
[               0.0],
[               0.0]])

...

Matrix([
[0.0],
[0.0],
[0.0],
[0.0]])

Matrix([
[0.0],
[0.0],
[0.0],
[0.0]])

Matrix([
[0.0],
[0.0],
[0.0],
[0.0]])

Matrix([
[0.0],
[0.0],
[0.0],
[0.0]])

Matrix([
[0.0],
[0.0],
[0.0],
[0.0]])